In [1]:
%cd /home/plent/Documenten/Gitlab/NeuralODEs/jax_neural_odes

from jaxkineticmodel.load_sbml.sbml_model import SBMLModel
import jax.numpy as jnp
import pandas as pd
import time
import numpy as np
import diffrax
import matplotlib.pyplot as plt
import jax
import amici
jax.config.update("jax_enable_x64", True)

/home/plent/Documenten/Gitlab/NeuralODEs/jax_neural_odes


In [2]:
model_names = ["Smallbone2013_SerineBiosynthesis","Becker_Science2010", "Fujita_SciSignal2010","Raia_CancerResearch","Messiha2013"]
# load model
n_reps = 10
model_names_pypesto_dir=["smallbone","becker","fujita","raia","messiha"]




timing_dict={}
for k, model_name in enumerate(model_names):
    # run to compile once
    filepath = "models/sbml_models/working_models/" + model_name + ".xml"
    model = SBMLModel(filepath)
    ts=jnp.linspace(0,10,500)
    kinetic_model=model.get_kinetic_model()
    kinetic_model.rtol=1e-12
    kinetic_model.atol=1e-12
    kinetic_model.dt0 = 1e-8

    S = model._get_stoichiometric_matrix()
    ys = kinetic_model(ts=ts,
                y0=model.y0,
                params=model.parameters)

    #run the kvaerno5 solver
    timings = []
    for i in range(n_reps):
        start= time.perf_counter()
        ys = kinetic_model(ts=ts,
                    y0=model.y0,
                    params=model.parameters)
        ys=pd.DataFrame(ys,columns=S.index)
        end = time.perf_counter()
        timings.append(end-start)

    print("mean", np.mean(timings)) #0.006830265596174286
    print("std", np.std(timings)) #0.004246095721556409
    timing_dict[f"{model_name}_kvaerno5"] = {'mean': np.mean(timings),'std': 2** np.std(timings)}

    #run the dopri5 solver
    kinetic_model._change_solver(solver=diffrax.Dopri5(), rtol=1e-12, atol=1e-12,dtol=1e-8,icoeff=0.1)

    ys = kinetic_model(ts=ts,
                y0=model.y0,
                params=model.parameters)


    timings = []
    for i in range(n_reps):
        start= time.perf_counter()
        ys = kinetic_model(ts=ts,
                    y0=model.y0,
                    params=model.parameters)
        ys=pd.DataFrame(ys,columns=S.index)
        end = time.perf_counter()
        timings.append(end-start)

    print("mean", np.mean(timings)) #0.006830265596174286
    print("std", np.std(timings)) #0.004246095721556409
    timing_dict[f"{model_name}_dopri5"] = {"mean": np.mean(timings), "std": np.std(timings)}

    #run the cvode solver from amici

        # filepath = "models/sbml_models/working_models/" + model_name + ".xml"
    output_dir = f"results/timing_benchmark/pypesto_compiled_models/{model_names_pypesto_dir[k]}"
    model_module = amici.import_model_module(model_name, output_dir)
    model = model_module.getModel()
    ts=np.linspace(0,10,500)
    model.setTimepoints(ts)
    solver = model.getSolver()
    solver.setAbsoluteTolerance(1e-12)
    solver.setRelativeTolerance(1e-12)
    solver.setSensitivityMethod(amici.SensitivityMethod_none)
    solver.setMaxSteps(maxsteps=300000)

    rdata = amici.runAmiciSimulation(model, solver)

    timings=[]
    for i in range(n_reps):
        start= time.perf_counter()
        rdata = amici.runAmiciSimulation(model, solver)

        end = time.perf_counter()
        timings.append(end-start)

    print("mean", np.mean(timings)) #0.006830265596174286
    print("std", np.std(timings)) #0.004246095721556409
    timing_dict[f"{model_name}_cvode"] = {"mean": np.mean(timings), "std": np.std(timings)}





17:22:05,247 - jax._src.xla_bridge - INFO - Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
17:22:05,252 - jax._src.xla_bridge - INFO - Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
mean 0.010133862102520652
std 0.0009166322431294692
mean 0.0036117053008638323
std 0.00019634109726631075
mean 0.002231114701135084
std 0.00015367312767491993
mean 0.005267181605449878
std 0.0006664866919553333
mean 0.002605088401469402
std 0.000231537925758812
mean 0.001977550904848613
std 0.0006342249539796311
mean 0.004584958901978098
std 0.00035002331665498406
mean 1.2348238781967666
std 0.11185014173636074
mean 0.0019684474042151122
std 0.0001281177354749127
mean 0.007477586306049488
std 0.0005347124075654328
mean 0.003978110098978504
std 0.00012371200162475756
mean 0.002549432296655141
std 0.0002276848986651051
mean 0.1286804362986004
std 0.02411886563

In [3]:
# Convert timing_dict to desired format
results = {}

for key, value in timing_dict.items():
    model, solver = key.rsplit("_", 1)
    if model not in results:
        results[model] = {}
    results[model][solver] = f"{value['mean']:.4f} ± {value['std']:.4f}"

df = pd.DataFrame(results).T  # Transpose to get models as rows
df.index.name = "Model"
df.columns.name = "Solver"

print(df)

# df.to_csv("results/timing_benchmark/time_benchmark_forward.csv")



Solver                                   kvaerno5           dopri5  \
Model                                                                
Smallbone2013_SerineBiosynthesis  0.0101 ± 1.0006  0.0036 ± 0.0002   
Becker_Science2010                0.0053 ± 1.0005  0.0026 ± 0.0002   
Fujita_SciSignal2010              0.0046 ± 1.0002  1.2348 ± 0.1119   
Raia_CancerResearch               0.0075 ± 1.0004  0.0040 ± 0.0001   
Messiha2013                       0.1287 ± 1.0169  0.3587 ± 0.0091   

Solver                                      cvode  
Model                                              
Smallbone2013_SerineBiosynthesis  0.0022 ± 0.0002  
Becker_Science2010                0.0020 ± 0.0006  
Fujita_SciSignal2010              0.0020 ± 0.0001  
Raia_CancerResearch               0.0025 ± 0.0002  
Messiha2013                       0.0184 ± 0.0008  


In [71]:
kinetic_model.max_steps

300000

In [123]:
## now lets look at pypesto/amici
import amici


for k, model_name in enumerate(model_names):

    # filepath = "models/sbml_models/working_models/" + model_name + ".xml"
    output_dir = f"results/timing_benchmark/pypesto_compiled_models/{model_names_pypesto_dir[k]}"
    model_module = amici.import_model_module(model_name, output_dir)
    model = model_module.getModel()
    ts=np.linspace(0,10,500)
    model.setTimepoints(ts)
    solver = model.getSolver()
    solver.setLinearSolver(amici.LinearSolver_dense)
    solver.setAbsoluteTolerance(1e-12)
    solver.setRelativeTolerance(1e-12)
    solver.setMaxSteps(maxsteps=300000)

    rdata = amici.runAmiciSimulation(model, solver)

    timings=[]
    for i in range(n_reps):
        start= time.perf_counter()
        rdata = amici.runAmiciSimulation(model, solver)

        end = time.perf_counter()
        timings.append(end-start)

    print(np.mean(timings))
    # print(np.std(timings))




0.003328111231850926
0.002086444035579916
0.0018909537083818576
0.0021350679741299243
0.02239957287395373
